In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!nvidia-smi

In [ ]:
# Ensure colab doesn't disconnect
%%javascript
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}setInterval(ClickConnect,60000)

In [ ]:
# Instalación de librerías 
!pip install keras==2.4.3
!pip install tensorflow==2.4.1

In [ ]:
#!unzip  "/content/drive/MyDrive/data_classfication_emo.zip (Unzipped Files)/test.zip" -d "/content/drive/MyDrive/data_classfication_emo.zip (Unzipped Files)"
!unzip "/content/drive/MyDrive/data_classfication_emo.zip (Unzipped Files)/train.zip" -d "/content/drive/MyDrive/data_classfication_emo.zip (Unzipped Files)"

In [ ]:
# Definimos las rutas del dataset 
train_data_dir = '/content/drive/MyDrive/data_classfication_emo.zip (Unzipped Files)/train'
val_data_dir = '/content/drive/MyDrive/data_classfication_emo.zip (Unzipped Files)/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.applications.imagenet_utils import preprocess_input


width_shape = 224
height_shape = 224
num_classes = 6
epochs = 25
batch_size = 32
class_names = ['Angry','fear','Happy','Neutral','Sad','Surprise']


train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(  
    train_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    #color_mode='grayscale',
    class_mode='categorical',shuffle=True)

val_generator = val_datagen.flow_from_directory(  
    val_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    #color_mode='grayscale',
    class_mode='categorical',shuffle=True)



In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input, AveragePooling2D,Activation
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
# example of loading the resnet50,mobilenetv3 model
#from tensorflow.keras.applications. import MobileNetV3Small
# load model
#Instantiating ResNet50,mobilenetv3
base_model_resnet = tf.keras.applications.MobileNetV3Small(weights='imagenet')

#Defining and Adding layers
model_resnet=Sequential()
#Add the Dense layers along with activation and batch normalization
model_resnet.add(base_model_resnet)
model_resnet.add(Flatten())

#Add the Dense layers along with activation and batch normalization
model_resnet.add(Dense(1024,activation=('relu'),input_dim=512))
model_resnet.add(Dense(512,activation=('relu'))) 
model_resnet.add(Dropout(0.4))
model_resnet.add(Dense(256,activation=('relu'))) 
model_resnet.add(Dropout(0.3))#Adding a dropout layer that will randomly drop 30% of the weights
model_resnet.add(Dense(128,activation=('relu')))
model_resnet.add(Dropout(0.2))
model_resnet.add(Dense(64,activation=('relu')))
model_resnet.add(Dropout(0.2))
model_resnet.add(Dense(6,activation=('softmax')))


opt = Adam(learning_rate=1e-4, decay=1e-4 / epochs)
model_resnet.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
# Configuración Tensorboard
from tensorflow.keras.callbacks import TensorBoard
import datetime, os

%load_ext tensorboard

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)

In [ ]:

model_resnet.fit(  
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_steps=val_generator.n//batch_size,
    #callbacks=[tensorboard_callback])
    )

In [ ]:
%tensorboard --logdir logs

In [ ]:
model_resnet.save("mode50new_mobinetv3Small_colab_final.h5")

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn import metrics
from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt

val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow_from_directory(  
    val_data_dir,
    target_size=(width_shape, height_shape),
    batch_size=batch_size,
    #color_mode='grayscale',
    class_mode='categorical',shuffle=False)

predictions = model_resnet.predict(val_generator)
y_pred = np.argmax(predictions, axis=1)
y_real = val_generator.classes

matc=confusion_matrix(y_real, y_pred)

plot_confusion_matrix(conf_mat=matc, figsize=(5,5), show_normed=False)
plt.tight_layout()

print(metrics.classification_report(y_real,y_pred, digits = 4))